In [ ]:
import time
import warnings
from functools import partial

import matplotlib as mpl
import numpy as np
from IPython import get_ipython
from IPython.display import Image

import pymor.tools.random

ip = get_ipython()
if ip is not None:
#    ip.run_line_magic('matplotlib', 'inline')
    ip.run_line_magic('matplotlib', 'ipympl')

warnings.filterwarnings('ignore', category=UserWarning, module='torch')

pymor.tools.random._default_random_state = None

mpl.rcParams['figure.facecolor'] = (1.0, 1.0, 1.0, 0.0)
#mpl.rcParams['figure.dpi'] = 200

from pymor.core.logger import getLogger, set_log_levels

set_log_levels({ # disable logging for some components
    'main': 'DEBUG',
    'pymor': 'WARN',
    'pymor.models': 'WARN',
    'pymor.discretizers.builtin': 'WARN',
    'pymor.discretizers.dunegdt': 'DEBUG',
    'pymor.analyticalproblems.functions.BitmapFunction': 'ERROR',
    'models.ann.ANNStateReductor': 'INFO',
    'models.vkoga.VkogaStateModel': 'INFO',
    'models.vkoga.VkogaStateReductor': 'DEBUG',
    'models.adaptive': 'DEBUG',
    'algorithms.optimization': 'DEBUG'})

logger = getLogger('main.main')

In [ ]:
# FOM
num_refines = 2
num_timesteps = 20

In [ ]:
from pymor.basic import *


def setup_problem_and_discretize(num_refines, nt):
    from spe10channel import discretize, make_problem

    grid, num_grid_elements, boundary_info, problem, parameter_space, mu_bar = make_problem(
        regime='diffusion dominated', num_global_refines=num_refines)

    fom, fom_data, coercivity_estimator = discretize(
        grid, num_grid_elements, boundary_info, problem, mu_bar, nt=nt)

    return parameter_space, fom, fom_data, coercivity_estimator


spatial_product = lambda m: m.energy_0_product

In [ ]:
logger.info('creating FOM:')
tic = time.perf_counter()

parameter_space, fom, fom_data, coercivity_estimator = setup_problem_and_discretize(
    num_refines, num_timesteps)

fom_offline_time = time.perf_counter() - tic
logger.info(f'  discretizing took {fom_offline_time}s')
logger.info(f'  grid has {fom_data["grid"].size(0)} elements,'
            f'FOM has {fom.solution_space.dim} DoFs, '
            f'uses {fom.time_stepper.nt} time steps')

logger.info(f'  input parameter space is {parameter_space.parameters.dim}-dimensional:')
logger.info(f'    {parameter_space}')

In [ ]:
logger.info('computing dual norm of output functional:')

assert not fom.output_functional.parametric
riesz_representative = spatial_product(fom).apply_inverse(fom.output_functional.as_vector())
dual_norm_output = np.sqrt(spatial_product(fom).apply2(riesz_representative, riesz_representative)[0][0])
del riesz_representative

logger.info(f'  {dual_norm_output}')

In [ ]:
from pymor.models.hierarchy import AdaptiveModelHierarchy

In [ ]:
rb_reductor = ParabolicRBReductor(fom, product=spatial_product(fom), coercivity_estimator=coercivity_estimator)

def reduction_rb(training_data, len_previous_training_data, models, reductors):
    U = fom.solution_space.empty(reserve=len(training_data))
    for _, u in training_data[len_previous_training_data:]:
        U.append(u)
    RB, _ = pod(U, product=fom.h1_0_semi_product)
    reductors[0].extend_basis(RB)
    # TODO: Use HaPOD here!!!
    return reductors[0].reduce()

def post_reduction_rb(training_data, models, reductors):
    return []

tolerance = 5e-3

# Settings for the two-stage hierarchy
models = [rb_reductor.reduce(), fom]
model_names = ['RB-ROM', 'FOM']
reductors = [rb_reductor]
reduction_methods = [reduction_rb]
post_reduction_methods = [post_reduction_rb]
training_frequencies = [1]

two_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                             training_frequencies, tolerance, visualizer=fom.visualizer,
                                             name='Two-stage model hierarchy')

In [ ]:
mu_init = fom.parameters.parse([9.5, 9.5])
mu_opt = fom.parameters.parse([3.1647, 11.0])

def objective_function(output, mu):
    return fom.T * np.mean(output) + 1/500 * mu["Da"][0]**2 - 1/10 * (mu["Pe"][0] - 9.)**2

In [ ]:
"""
num_params = 20
uniform_params = parameter_space.sample_uniformly(num_params)
res = np.zeros((num_params, num_params))
for i, mu in enumerate(uniform_params):
    output = fom.output(mu).flatten()
    res[(i-(i%num_params))//num_params, i%num_params] = objective_function(output, mu)

import matplotlib.pyplot as plt
plt.close()
fig, axs = plt.subplots()
mp = axs.imshow(res.T, origin='lower')
fig.colorbar(mp, ax=axs)
plt.show()
"""

In [ ]:
"""
res_alt = res.copy()
params_array = np.array([p.to_numpy() for p in uniform_params]).reshape((num_params, num_params, 2))
s = 1/500
t = 1/10
res_alt += s * params_array[..., 0]**2 - t * (params_array[..., 1] - 9.)**2
#print(f"Matrix of output values: {res_alt}")
print(f"Minimum value: {np.min(res_alt)}")
amin = np.unravel_index(res_alt.argmin(), res_alt.shape)
print(f"Minimum index: {amin}")
print(f"Minimum parameter: {uniform_params[amin[0]*num_params+amin[1]]}")
plt.close()
fig_alt, axs_alt = plt.subplots(1, 2)
mp = axs_alt[0].imshow(res.T, origin='lower')
fig_alt.colorbar(mp, ax=axs_alt[0])
mp_alt = axs_alt[1].imshow(res_alt.T, origin='lower')
fig_alt.colorbar(mp_alt, ax=axs_alt[1])
plt.show()
"""

In [ ]:
"""
fig = plt.figure(frameon=False)
fig.set_size_inches(1, 1)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(res.T, origin='lower', aspect='auto')
fig.savefig('fig_optimization_background_new.png', dpi=100)
"""

In [ ]:
mean = (5.005, 10)
cov = [[2, 0], [0, 1]]

from scipy.stats import multivariate_normal
rv = multivariate_normal(mean, cov)

def random_sampling_function():
    return rv.rvs()

density_function_monte_carlo = rv.pdf

In [ ]:
from pymor.models.interact import interact_model_hierarchy

In [ ]:
from pymor.reductors.ml import MachineLearningInstationaryReductor

method = 'gpr'
model_parameters = {}

ml_reductor = MachineLearningInstationaryReductor(fom=fom, training_set=None, validation_set=None, method=method, model_parameters=model_parameters, product=fom.h1_0_semi_product)

def reduction_ml(training_data, len_previous_training_data, models, reductors):
    rb_rom = models[1]
    rb_reductor = reductors[1]
    ml_reductor = reductors[0]
    error_estimator = rb_rom.error_estimator
    ml_reductor.reduced_basis = rb_reductor.bases['RB']
    red_dim = len(ml_reductor.reduced_basis)
    training_data = [(mu, np.pad(dat.to_numpy()[0], (0, red_dim - len(dat.to_numpy()[0])),
                                 mode='constant', constant_values=0.))
                     for (mu, dat) in training_data if np.linalg.norm(dat.to_numpy().flatten()) > 1e-10]
    ml_reductor.training_data = training_data
    ml_rom = ml_reductor.reduce()
    return ml_rom.with_(error_estimator=error_estimator, T=rb_rom.T, time_stepper=rb_rom.time_stepper)

def post_reduction_ml(training_data, models, reductors):
    return []

def post_reduction_rb(training_data, models, reductors):
    return [models[0]]
    #return [reduction_ml(training_data[1], None, models, reductors)]

models = [None, rb_reductor.reduce(), fom]
model_names = ['ML-ROM', 'RB-ROM', 'FOM']
reductors = [ml_reductor, rb_reductor]
reduction_methods = [reduction_ml, reduction_rb]
post_reduction_methods = [post_reduction_ml, post_reduction_rb]
training_frequencies = [10, 1]

three_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                               training_frequencies, tolerance)

# Katalytische Filter

Wir betrachten eine parametrisierte lineare Advektions-Diffusions-Reaktions-Gleichung in einem künstlichen Leitungsrohr $\Omega=[0,5]\times[0,1]=\Omega_\mathrm{w}\cup\Omega_\mathrm{c}$, aufgeteilt in einen Washcoat $\Omega_\mathrm{w}=[0,5]\times[0,h_\mathrm{w}]$ der Höhe $h_\mathrm{w}=0.34$ und einen Kanal $\Omega_\mathrm{c}=\Omega\setminus\Omega_\mathrm{w}$ mit Einflussrand $\Gamma_\mathrm{in}=\{x=0\}\times[h_\mathrm{w},1]$ und Ausflussrand $\Gamma_\mathrm{out}=\{x=5\}\times[h_\mathrm{w},1]$. Für jeden Parameter $\mu=(\mathrm{Da},\mathrm{Pe})\in\mathcal{P}$ aus der Parametermenge $\mathcal{P}=[0.01,10]\times[9,11]$ suchen wir die Lösung $u(\mu)\in L^2(0,T;V_g)$ mit $\partial_tu(\mu)\in L^2(0,T;V_0')$ der Gleichung für $T=5$:

$$
    \partial_t u(\mu)-\nabla\cdot(\kappa\nabla u(\mu))+\mathrm{Pe}\nabla\cdot(\vec{v}u(\mu))+\mathrm{Da}\chi_{\Omega_\mathrm{w}}u(\mu)=0\qquad\text{in }\Omega\times(0,T)
$$

mit Neumann-Randbedingung $(\kappa\nabla u(\mu))\cdot n_{\partial \Omega}=0$ auf $\Gamma_\mathrm{out}$, Dirichlet-Randbedingung $u(\mu)=g=\chi_{\Gamma_\mathrm{in}}$ auf $\partial\Omega\setminus\Gamma_\mathrm{out}$ und Anfangsbedingung $u(t=0;\mu)=\chi_{\Gamma_\mathrm{in}}$. Das Advektionsfeld ist definiert als $\vec{v}=(\chi_{\Omega_\mathrm{c}},0)^\top$ und die Diffusion $\kappa$ ist gegeben durch $1$ im Kanal $\Omega_\mathrm{c}$ beziehungsweise durch die erste Komponente des Permeabilitätsfeldes des Spe10 Modells im Washcoat $\Omega_\mathrm{w}$.

Das Leitungsrohr mit Kanal und Washcoat ist unten abgebildet.

In [ ]:
Image("channel_washcoat.png", width=1000)

Ziel ist es, einen möglichst geringen Ausfluss der Schadstoffs am Ausflussrand bei gleichzeitig möglichst großem Durchsatz zu erreichen. Es soll also möglichst viel des kontaminierten Stoffes in das Rohr geleitet werden (vom Einfluss- zum Ausflussrand) und dabei möglichst viel des Schadstoffes entfernt werden (durch die Reaktion mit dem Filtermaterial im Washcoat). Allerdings sind die Kosten zur Konstruktion eines besonders effizienten Filters (das heißt mit besonders starker Reaktion, also einem großen Wert der Damköhler-Zahl $\mathrm{Da}$) hoch. Dies gilt es beim Design des Filters zu berücksichtigen. Zusammenfassend soll das folgende Optimierungsproblem mit der Zielfunktion $\mathcal{J}\colon\mathcal{P}\to\mathbb{R}$ gelöst werden:

$$
    \min_{\mu\in\mathcal{P}} \mathcal{J}(\mu),\qquad\qquad\mathcal{J}(\mu):=\int\limits_{0}^{T}\underbrace{\left(\frac{1}{|\Gamma_\mathrm{out}|}\int\limits_{\Gamma_\mathrm{out}}u(t;\mu)\,\mathrm{d}s\right)}_{\text{Schadstoff am Ausfluss zum Zeitpunkt }t}\,\mathrm{d}t + \underbrace{\frac{\mathrm{Da}^2}{500}}_{Produktionskosten} - \underbrace{\frac{(\mathrm{Pe} - 9)^2}{10}}_{Durchsatz}.
$$

# Adaptive Modellhierarchien

Um möglichst effiziente Filter zu bauen, sind Simulationen des Verhaltens verschiedener Filter mit verschiedenen Wahlen der Parameter $\mu=(\mathrm{Da},\mathrm{Pe})\in\mathcal{P}$ notwendig. Die gegebene partielle Differentialgleichung wird hierzu zunächst mit der Methode der Finiten Elemente diskretisiert. Dadurch erhalten wir das sogenannte Full order model (FOM). Dieses liefert sehr genaue Lösungen, allerdings ist die Berechnung der Lösung für einen einzelnen bereits recht aufwendig. Mithilfe geeigneter Trainingsdaten können wir jedoch ein reduziertes Modell, das RB-ROM, konstruieren, das eine näherungsweise Lösung berechnet und deutlich schneller als das FOM ist. Dieses RB-ROM können wir zudem durch Ansätze aus dem maschinellen Lernen ergänzen, was uns insgesamt das ML-ROM liefert.

Für das RB-ROM und das ML-ROM existiert ein Fehlerschätzer $\eta_\mu$, den wir nutzen können, um eine obere Schranke für den Approximationsfehler gegenüber der Lösung des FOM zu bestimmen. Wir möchten bei unseren Simulationen nur solche Lösungen akzeptieren, deren Fehler kleiner als eine vorgegebene Toleranz beziehungsweise Rechengenauigkeit sind. Das RB-ROM und das ML-ROM können automatisch mit zusätzlichen Daten des FOM trainiert werden, um die entsprechende Rechengenauigkeit zu erreichen.

Die verschiedenen Modelle können in einer adaptiven Modellhierarchie kombiniert werden, um ihre individuellen Vorteile hinsichtlich Rechenzeit und Genauigkeit bestmöglichst auszunutzen. Insgesamt ergibt sich ein effizientes Zusammenspiel der verschiedenen Verfahren, die sich insbesondere gegenseitig zusätzliche Trainingsdaten liefern und sich daher gegenseitig verbessern. Der Einsatz des Fehlerschätzers ermöglicht zudem, eine Garantie für die Rechengenauigkeit abzugeben. Der Fehler der berechneten (approximativen) Lösung wird also immer unterhalb der vorgegebenen Toleranz liegen, notfalls wird auf das FOM zurückgegriffen, um die gewünschte Genauigkeit sicherzustellen.

<style>.myst, .myst p {font-size: 20pt;}</style>

## Überblick über die verschiedenen Modelle

| Modell | Bezeichnung für die Lösung zum Parameter $\mu$ | Genauigkeit | Rechenaufwand | Sonstiges |
|--------|-------------|---------------|-----------|-----------|
| FOM    | $u(\mu)$ | Hinreichend genau für alle Parameter (Referenzlösung) | Sehr aufwendig | - |
| RB-ROM | $u^N(\mu)$ | Etwas ungenauer als das FOM | Deutlich schneller als das FOM | Effiziente Fehlerschätzung möglich |
| ML-ROM | $\tilde{u}^N(\mu)$ | Ungenauer als das RB-ROM | Schneller als das RB-ROM | Fehlerschätzung vom RB-ROM kann wiederverwendet werden |

## Visualisierung der Modellhierarchie

In [ ]:
Image("model_hierarchy.png", width=1000)

## Anwendungsszenarien in der interaktiven Simulation

In der Simulation unten werden verschiedene Anwendungsszenarien betrachtet:

### Manuelle Parameterwahl

Durch Anklicken des entsprechenden Punkts im Parameterraum, kann der entsprechende Parameter manuelle gewählt werden und die adaptive Modellhierarchie wird automatisch für den gewählten Parameter aufgerufen und die Lösung berechnet. Des Weiteren wird der zugehörige Zielfunktionswert für die Kostenfunktion in einem Graphen aufgetragen.

### Parameteroptimierung

Wie oben beschrieben, möchten wir den Parameter bestimmen, der den besten Kosten-Nutzen-Wert liefert, das heißt das beste Verhältnis zwischen Filterleistung, Durchsatz und Herstellungskosten darstellt. Hierzu kann ein Optimierungsalgorithmus genutzt werden, der auf mehrfacher Auswertung der Zielfunktion für verschiedene Parameterwerte basiert. Die Auswertung der Zielfunktion für einen gegebenen Parameter benötigt die Lösung der Differentialgleichung, welche mittels der adaptiven Modellhierarchie näherungsweise bestimmt wird. Die Modellhierarchie kann also eingebettet in einen Optimierungsalgorithmus genutzt werden.

### Abschätzung der Auswirkungen von Materialunsicherheiten

Typischerweise ist es sehr schwierig, Filtermaterialien mit exakt den vorgegebenen Eigenschaften herzustellen. Selbst wenn ein optimaler Parameter gefunden ist, wird der tatsächlich produzierte Filter kleinere Abweichungen vom gewünschten Verhalten aufweisen. Um diese Abweichungen zu untersuchen, kann eine sogenannte Monte Carlo Simulation genutzt werden, bei der der durchschnittliche Zielfunktionswert für eine gegebene Verteilung der auftretenden Parameter mit Hilfe von einer großen Zahl an Simulationen geschätzt wird. Hierbei wird der durchschnittliche Zielfunktionswert einfach als Mittel über die Ergebnisse sämtlicher Simulationen angenähert. Werden viele Simulationen durchgeführt, so liefert dieses Verfahren eine gute Approximation des tatsächlich gesuchten Erwartungswertes.

In [ ]:
interact_model_hierarchy(three_stage_hierarchy, parameter_space, model_names,
                         output_function=objective_function, objective_function=objective_function, initial_parameter=mu_init, optimal_parameter=mu_opt, optimization_bg_image="fig_optimization_background_new.png", optimization_bg_image_limits=(0.0278514264616242, 0.000111348743739251), show_solution=True, visualizer=fom.visualizer, random_sampling_function=random_sampling_function, density_function_monte_carlo=density_function_monte_carlo, solution_plot_extent=(0, 2.5, 0, 1), language='de',
                         fig_width=17)